In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
import json
import cv2
import numpy as np
from PIL import Image
import pycocotools.mask as maskUtils

from matting_api import Matting

sys.path.append('..')
import utils
import inference as infer
from datasets import COCOADataset, read_COCOA
from demo_utils import *

font = {'family' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

In [86]:
# [Right, Bottom]
def neighbors(r, c, R, C):
    lst = []
    if c + 1 < C:
        lst.append((r, c + 1))
    else:
        lst.append(None)
    if r + 1 < R:
        lst.append((r + 1, c))
    else:
        lst.append(None)
    return lst

In [99]:
import cv2

def pad_mask(mask, size=9):
    kernel = np.ones((size, size))
    return cv2.filter2D(mask, 1, kernel)

In [101]:
from tqdm import tqdm

channels = 2

phase = 'train'
root_dict = {'train': "../data/COCOA/train2014", 'val': "../data/COCOA/val2014"}

img_root = root_dict[phase]
annot_path = "../data/COCOA/annotations/COCO_amodal_{}2014.json".format(phase)

output_root = "../data/COCOA/pixel_graphs_{}/".format(phase)

data_reader = COCOADataset(annot_path)
img_count = len(data_reader.annot_info)

for img_idx in tqdm(range(img_count)):
    modal, category, ori_bboxes, amodal_gt, image_fn = data_reader.get_image_instances(
        img_idx, with_gt=True, ignore_stuff=True)
    
    modal_count = len(modal)
    gt_order_matrix = infer.infer_gt_order(modal, amodal_gt)
    
    image_fn = os.path.join(img_root, image_fn)
    img = Image.open(image_fn)
    height, width = img.height, img.width
    image = np.array(img)
    
    # Construct modals and graph
    modals = np.zeros((height, width))
    for i in range(modal_count):
        modals += (i + 1) * (modal[i] > 0)
    graph = np.zeros((height, width, channels))

    for r in range(height):
        for c in range(width):
            pairs = neighbors(r, c, height, width)
            for k in range(channels):
                pair = pairs[k]
                if not pair:
                    continue
                u, v = pair
                i, j = int(modals[r, c]), int(modals[u, v])
                if not i and j:
                    graph[r, c, k] = -1
                elif not j and i:
                    graph[r, c, k] = 1
                elif not j and not i:
                    graph[r, c, k] = 0
                else:
                    graph[r, c, k] = gt_order_matrix[i - 1, j - 1]
    
    # Save graph
    path = os.path.join(output_root, str(img_idx) + '_graph')
    tensor = torch.tensor(graph)
    torch.save(tensor, path)
    
    # Save mask
    mask = pad_mask(1 * (modals > 0)) > 0
    path = os.path.join(output_root, str(img_idx) + '_mask')
    tensor = torch.tensor(mask)
    torch.save(tensor, path)

100%|██████████| 2500/2500 [24:28<00:00,  1.70it/s]


In [115]:
path = os.path.join(output_root, str(0) + '_graph')
graph = torch.load(path)

path = os.path.join(output_root, str(0) + '_mask')
mask = torch.load(path)

In [130]:
import cvbase as cvb

In [132]:
data = cvb.load(annot_path)

In [133]:
data['images']

[{'license': 1,
  'file_name': 'COCO_train2014_000000000321.jpg',
  'coco_url': 'http://mscoco.org/images/321',
  'height': 480,
  'width': 640,
  'date_captured': '2013-11-20 12:36:25',
  'flickr_url': 'http://farm5.staticflickr.com/4096/4750559893_49fb0baf7f_z.jpg',
  'id': 321},
 {'license': 1,
  'file_name': 'COCO_train2014_000000000612.jpg',
  'coco_url': 'http://mscoco.org/images/612',
  'height': 480,
  'width': 640,
  'date_captured': '2013-11-18 11:30:40',
  'flickr_url': 'http://farm3.staticflickr.com/2735/4198283385_aaa66e690c_z.jpg',
  'id': 612},
 {'license': 3,
  'file_name': 'COCO_train2014_000000001108.jpg',
  'coco_url': 'http://mscoco.org/images/1108',
  'height': 453,
  'width': 640,
  'date_captured': '2013-11-20 06:28:28',
  'flickr_url': 'http://farm3.staticflickr.com/2753/4145840774_309935ac81_z.jpg',
  'id': 1108},
 {'license': 1,
  'file_name': 'COCO_train2014_000000001145.jpg',
  'coco_url': 'http://mscoco.org/images/1145',
  'height': 480,
  'width': 640,
  '